# IMDB Review Sentiment Analysis
This should explain the problem with some links

In [64]:
import pandas as pd
import numpy as np
from glob import glob
import os

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers.embeddings import Embedding

In [65]:
# Checking if we have the data. If not download it
# This needs to be done...
# Mention that we could use the built-in keras.load_imdb() but want to do it manually for clarity
vocab_path = 'data/imdb.vocab'
train_path = 'data/train'
test_path = 'data/test'

VOCAB_SIZE = 20
NUM_SAMPLES = 100
MAX_REVIEW_LEN = 50

In [ ]:
# train_file_paths = glob(os.path.join(train_path, 'neg', '*'))
# print(train_file_paths[0])
# with open(train_file_paths[0]) as file:
#     data = file.read()

In [ ]:
# Using the built-in vocab
# with open(vocab_path) as file:
#     vocab = file.read()
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(vocab)

In [ ]:
# data_sample = tokenizer.texts_to_sequences(data)
# print(data_sample)

# print(len(data))
# print(len(data_sample))

In [ ]:
# Get an array of all the filepaths

In [66]:
train_files = {}
train_files['pos'] = glob(os.path.join(train_path, 'pos', '*.txt'))
train_files['neg'] = glob(os.path.join(train_path, 'neg', '*.txt'))
# train_files['unsup'] = glob(os.path.join(train_path, 'unsup', '*.txt'))

In [ ]:
# This needs to get the indices for training and validation


In [67]:
# Turn this into a function at some point soon so we can get train, test, val
# This might take a while, needs to read in a large amount of data
# Later read in the testing data as well (write a function)
x_train = []
y_train = []
sentiment_map = {'pos': 1, 'neg': 0}
for sentiment in train_files:
    print(len(train_files[sentiment]))
    # Do the shuffle here for indices
    # Try different splits
    for file_name in train_files[sentiment][0:NUM_SAMPLES]:
        temp_ = []
        with open(file_name) as file_:
            temp_ = file_.read()
        x_train.append(temp_)
        y_train.append(sentiment_map[sentiment])

print(len(x_train), len(y_train))

12500
12500
200 200


In [68]:
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(x_train, maxlen=MAX_REVIEW_LEN)

In [59]:
# create the model

# Look into these hyper-parameters
# eventually will use a sweep
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(VOCAB_SIZE, embedding_vecor_length, input_length=MAX_REVIEW_LEN))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 32)            640       
_________________________________________________________________
dropout_5 (Dropout)          (None, 50, 32)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_6 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 53,941
Trainable params: 53,941
Non-trainable params: 0
_________________________________________________________________
None


In [60]:
model.fit(x_train, y_train, nb_epoch=3, batch_size=64)

/home/mike/Development/personal/sentiment_analysis/venv/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/3
200/200 [==============================] - 1s 3ms/step - loss: 0.6935 - acc: 0.4850
Epoch 2/3
200/200 [==============================] - 0s 546us/step - loss: 0.6923 - acc: 0.5250
Epoch 3/3
200/200 [==============================] - 0s 595us/step - loss: 0.6917 - acc: 0.5850


In [62]:
# x_test, y_test = get_encoded_x_y()

In [61]:
# scores = model.evaluate(X_test, y_test, verbose=0)
# print("Accuracy: %.2f%%" % (scores[1]*100))